<a href="https://colab.research.google.com/github/lichkim/VQE_Summer_Internship/blob/master/tutorial_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install required packages
!pip install -U nlopt
import numpy as np
import tensorflow as tf
import nlopt
from itertools import product
import logging

In [ ]:
class Sigma:

    '''
    Sigma class generates algebraic primitives required to compute the system of implicit ODEs for Euler angles.
    '''

    def __init__(self, *args):
            self.string = [i for i in args]
            self.order = len(args)
            self.digit = sum([args[i]*4**(self.order-i-1) for i in range(len(args))])
            self.pauli = tf.constant([
                 [[-1j/2,0],[0,-1j/2]],[[0, 1j/2],[1j/2, 0]],[[0, -1/2],[1/2, 0]],[[1j/2, 0],[0, -1j/2]]
                      ],dtype=tf.complex64,shape=(4,2,2))
            self.__struct_const_0 = tf.constant(
                [
                    np.zeros([4,4]),
                    [[0.0, 0.0, 0.0, 0.0],[0.0, 0.0, 0.0, 0.0],[0.0, 0.0, 0.0, -1.0],[0.0, 0.0, 1.0, 0.0]],
                    [[0.0, 0.0, 0.0, 0.0],[0.0, 0.0, 0.0, 1.0],[0.0, 0.0, 0.0, 0.0],[0.0, -1.0, 0.0, 0.0]],
                    [[0.0, 0.0, 0.0, 0.0],[0.0, 0.0, -1.0, 0.0],[0.0, 1.0, 0.0, 0.0],[0.0, 0.0, 0.0, 0.0]]
                    ], dtype=tf.complex64
            )
            self.__struct_const_1 = tf.constant(
                [
                    -1j*np.eye(4),
                    -1j*np.array([[0.0, 1.0, 0.0, 0.0],[1.0, 0.0, 0.0, 0.0],[0.0, 0.0, 0.0, 0.0],[0.0, 0.0, 0.0, 0.0]]),
                    -1j*np.array([[0.0, 0.0, 1.0, 0.0],[0.0, 0.0, 0.0, 0.0],[1.0, 0.0, 0.0, 0.0],[0.0, 0.0, 0.0, 0.0]]),
                    -1j*np.array([[0.0, 0.0, 0.0, 1.0],[0.0, 0.0, 0.0, 0.0],[0.0, 0.0, 0.0, 0.0],[1.0, 0.0, 0.0, 0.0]])
                    ], dtype=tf.complex64
            )
            self.__vector = tf.constant([
                [1,0,0,0],
                [0,1,0,0],
                [0,0,1,0],
                [0,0,0,1]], dtype=tf.float32
            )
            self.index_perm = list(range(0,self.order*2,2)) + list(range(1,self.order*2+1,2))
            self.fundamental = self.rep_fundamental()
            self.adjoint = self.rep_adjoint()
            self.fundamental_z = tf.eye(2,dtype=tf.complex64)
            for i in range(1,self.order):
                self.fundamental_z = tf.tensordot(self.fundamental_z,tf.eye(2,dtype=tf.complex64),axes=0)
            self.fundamental_z = tf.transpose(self.fundamental_z,perm=self.index_perm)
            self.adjoint_z = tf.eye(4,dtype=tf.float32)
            for i in range(1,self.order):
                self.adjoint_z = tf.tensordot(self.adjoint_z,tf.eye(4,dtype=tf.float32),axes=0)
            self.adjoint_z = tf.transpose(self.adjoint_z,perm=self.index_perm)
            self.adjoint_square = tf.tensordot(self.adjoint,self.adjoint,axes=self.order)
            self.adj_vec = self.adj_vector()

    def rep_fundamental(self):
        index = self.string
        ret = self.pauli[index[0]]
        for i in range(1,len(index)):
            ret = 2j*tf.tensordot(ret,self.pauli[index[i]],axes=0)
        return tf.transpose(ret,perm=self.index_perm)


    def rep_adjoint(self):
        return tf.transpose(tf.cast(self.__rep_adjoint(self.string),dtype=tf.float32),perm=self.index_perm)

    def __rep_adjoint(self,sigma,center=False,grade=False):
        order = len(sigma)
        if grade:
            ret = self.__struct_const_1[sigma[0]]
        else:
            ret = self.__struct_const_0[sigma[0]]

        if order > 1:
            if grade:
                ret = 1.j* (
                    tf.tensordot(self.__rep_adjoint(sigma[0:order-1],center=True,grade=True),  self.__rep_adjoint([sigma[order-1]],center=True,grade=True),axes=0) +
                    tf.tensordot(self.__rep_adjoint(sigma[0:order-1],center=True,grade=False), self.__rep_adjoint([sigma[order-1]],center=True,grade=False),axes=0)
                    )
            else:
                ret = 1.j* (
                    tf.tensordot(self.__rep_adjoint(sigma[0:order-1],center=True,grade=True),  self.__rep_adjoint([sigma[order-1]],center=True,grade=False),axes=0) +
                    tf.tensordot(self.__rep_adjoint(sigma[0:order-1],center=True,grade=False), self.__rep_adjoint([sigma[order-1]],center=True,grade=True),axes=0)
                    )


        return ret

    def adj_vector(self):
        index = self.string
        ret = self.__vector[index[0]]
        for i in range(1,len(index)):
            ret = tf.tensordot(ret,self.__vector[index[i]],axes=0)
        return ret
    @tf.function
    def exp_fundamental(self,x):
        s = tf.cast(tf.math.sin(x/2),tf.complex64)
        c = tf.cast(tf.math.cos(x/2),tf.complex64)
        return tf.tensordot(c, self.fundamental_z,axes=0) + tf.tensordot(2*s,  self.fundamental,axes=0)
    @tf.function
    def exp_adjoint(self,x):
        return self.adjoint_z + tf.tensordot(tf.math.sin(x), self.adjoint,axes=0) + tf.tensordot((1 - tf.math.cos(x)), self.adjoint_square,axes=0)

In [ ]:
s1 = Sigma(1,2)
s1.fundamental

<tf.Tensor: shape=(2, 2, 2, 2), dtype=complex64, numpy=
array([[[[ 0. +0.j,  0. +0.j],
         [ 0. +0.j,  0.5+0.j]],

        [[ 0. +0.j,  0. +0.j],
         [-0.5+0.j,  0. +0.j]]],


       [[[ 0. +0.j,  0.5+0.j],
         [ 0. +0.j,  0. +0.j]],

        [[-0.5+0.j,  0. +0.j],
         [ 0. +0.j,  0. +0.j]]]], dtype=complex64)>

In [ ]:
class Euler():
    '''
    Euler class genetates various list of indices of sigma strings of the rank n.
    '''
    def __init__(self, order):
        self.order = order
        self.basis = np.array(list(product([0,1,2,3], repeat = order)))
        self.toral = np.array(list(product([0,3], repeat = order)))
        self.kprime = self.__kprime(order)
        self.aprime = self.__aprime(order)
        self.k = self.__k(order)
        self.a = self.__a(order)
        self.kak = np.concatenate((self.k,self.a,self.k),axis=0)
        self.generator = self.__generator(order)
        self.dim = len(self.kak)

    def __kak(self,order):
        if order == 1:
            return np.array([[3],[1],[3]])
        else:
            # return self.__a(order-1)
            return np.concatenate((self.__k(order),self.__a(order),self.__k(order)),axis=0)

    def __kprime(self,order):
        if order == 1:
            return np.array([],dtype=np.int32)
        else:
            # print(self.__kak(order-1))
            return np.insert(self.__kak(order-1),0,0,axis=1)

    def __aprime(self,order):
        if order == 1:
            return np.array([],dtype=np.int32)
        else:
            return np.insert(np.array(list(product([0,3], repeat = order-1))),0,3,axis=1)

    def __k(self,order):
        if order == 1:
            return np.array([[3]])
        else:
            # print(self.kprime)
            # return np.concatenate((self.kprime,self.aprime,self.kprime),axis=0)
            return np.concatenate((self.__kprime(order),self.__aprime(order),self.__kprime(order)),axis=0)

    def __a(self,order):
        if order == 1:
            return np.array([[1]])
        else:
            return np.insert(np.array(list(product([0,3], repeat = order-1))),0,1,axis=1)

    def __generator(self,order):
        if order == 1:
            return np.array([[1],[3]])
        else:
            ret = np.insert(self.__generator(order-1),0,0,axis=1)
            return np.concatenate((ret,self.__a(order),self.__aprime(order)),axis=0)

In [ ]:
def generate_first_kind(eu):
  '''
  generates a closure relating canonical coordinates of the first kind to an element in SU(2^n) with respect to sigma strings.
  The argument is an instance of the Euler class.
  '''
  eu_fund = tf.constant(np.array([tf.reshape(Sigma(*s).fundamental,[2**eu.order,2**eu.order]) for s in eu.basis]),dtype=tf.complex128);
  dim = len(eu.basis)
  @tf.function
  def first_kind(theta):
    '''
    returns an element of SU(2^n) from given canonical coordinates of the first kind whose generators are sigma strings.
    '''
    theta = tf.cast(theta,tf.complex128)
    return tf.linalg.expm(tf.tensordot(theta,eu_fund,axes=1))
  return first_kind

def generate_second_kind(eu):
  '''
  generates a closure relating canonical coordinates of the second kind to an element in SU(2^n) with respect to sigma strings.
  The argument is an instance of the Euler class.
  '''
  eu_fund = tf.constant(np.array([tf.reshape(Sigma(*s).fundamental,[2**eu.order,2**eu.order]) for s in eu.basis]),dtype=tf.complex128);
  dim = len(eu.basis)
  identity = tf.eye(2**eu.order,dtype=tf.complex128)
  @tf.function
  def second_kind(theta):
    '''
    returns an element of SU(2^n) from given canonical coordinates of the second kind whose generators are sigma strings.
    '''
    c = tf.cast(tf.cos(theta/2),tf.complex128)
    s = tf.cast(2*tf.sin(theta/2),tf.complex128)
    ret = c[0] * identity + s[0] * eu_fund[0]
    for i in range(1,dim):
      ret = tf.matmul(ret,(c[i] * identity + s[i] * eu_fund[i]))
    return ret
  return second_kind

In [ ]:
def diff_sum(mat1,mat2):
  '''
  sums over the absolute square elementwise difference between two complex matrices.
  '''
  diff = mat1 - mat2
  return tf.cast(tf.reduce_sum(diff*tf.math.conj(diff)),tf.float64)

def generate_diff_sum_grad_fk(mat):
  '''
  generates a function closure computing the diff_sum for a first kind matrix and a complex matrix as well as its gradient.
  '''
  order = tf.cast(tf.experimental.numpy.log2(tf.cast(mat.shape[0],tf.float32)),tf.int16)
  eu = Euler(order)
  fk = generate_first_kind(eu)

  def diff_sum_grad_fk(theta):
    '''
    returns the diff_sum of a first kind matrix and a complex matrix with its gradient.
    The first element of the return is the gradient and the last element is the value of diff_sum.
    '''
    with tf.GradientTape() as tape:
      tape.watch(theta)
      o = diff_sum(mat,fk(theta))

    return tape.gradient(o,theta),o

  return diff_sum_grad_fk

def generate_diff_sum_grad_sk(mat):
  '''
  generates a function closure computing the diff_sum for a second kind matrix and a complex matrix as well as its gradient.
  '''
  order = tf.cast(tf.experimental.numpy.log2(tf.cast(mat.shape[0],tf.float32)),tf.int16)
  eu = Euler(order)
  sk = generate_second_kind(eu)
  def diff_sum_grad_sk(theta):
    '''
    returns the diff_sum of a second kind matrix and a complex matrix with its gradient.
    The first element of the return is the gradient and the last element is the value of diff_sum.
    '''
    with tf.GradientTape() as tape:
      tape.watch(theta)
      o = diff_sum(mat,sk(theta))

    return tape.gradient(o,theta),o

  return diff_sum_grad_sk

In [ ]:
# Optimization begins.
def generate_obj_fk(mat):
  '''
  generates an objective function for the optimization problem to find the canonical coordinates of the first kind based on the diff_sum.
  '''
  order = tf.cast(tf.experimental.numpy.log2(tf.cast(mat.shape[0],tf.float32)),tf.int16)
  eu = Euler(order)
  fk_grad = generate_diff_sum_grad_fk(mat)
  def obj_fk(x,grad):
    tf_x = tf.constant(x,dtype=tf.float64)
    if grad.size > 0:
      obj = fk_grad(tf_x)
      for i in range(grad.size):
        grad[i] = obj[0].numpy()[i]

    return obj[1].numpy()

  return obj_fk

def generate_obj_sk(mat):
  '''
  generates an objective function for the optimization problem to find the canonical coordinates of the first kind based on the diff_sum.
  '''
  order = tf.cast(tf.experimental.numpy.log2(tf.cast(mat.shape[0],tf.float32)),tf.int16)
  eu = Euler(order)
  sk_grad = generate_diff_sum_grad_sk(mat)
  def obj_sk(x,grad):
    tf_x = tf.constant(x,dtype=tf.float64)
    if grad.size > 0:
      obj = sk_grad(tf_x)
      for i in range(grad.size):
        grad[i] = obj[0].numpy()[i]

    return obj[1].numpy()

  return obj_sk

In [ ]:
# unitary matrices to be coordinated
h_gate = tf.cast(tf.constant([[1.,1.],[1.,-1.]])/tf.sqrt(2.),dtype=tf.complex128); # Hadarmard gate
s_gate = tf.constant([[1,0],[0,-1.0j]],dtype=tf.complex128); # S gate
t_gate = tf.constant(np.array([[1,0],[0,np.exp(1.0j*np.pi/8)]],dtype=np.complex128)); # T gate
cnot_gate = tf.constant(np.array([[1,0,0,0],[0,1,0,0],[0,0,0,1],[0,0,1,0]],dtype=np.complex128)); #CNOT gate

In [ ]:
unitary_target = h_gate
dim = tf.reduce_prod(unitary_target.shape).numpy().astype(int).item()

In [ ]:
logging.getLogger('tensorflow').setLevel(logging.ERROR) # quiets Tensorflow warnings.
obj_fk = generate_obj_fk(unitary_target) # generates an objective function for a target unitary matrix.
opt = nlopt.opt(nlopt.LD_LBFGS,dim) # chooses an optimization algorithm.
opt.set_min_objective(obj_fk) # sets the objective function for the chosen optimization algorithm.
opt.set_xtol_rel(1e-5) # sets the tolerance for the convergence.
theta_opt_fk = opt.optimize(np.ones(dim)) # sets the initial varaibles.
obj_final = opt.last_optimum_value() # assigns the variables of the last iteration.
# prints results of the optimization algorithm
print("minimum at",theta_opt_fk)
print("the objective function converges to", obj_final)
print("optimization result: ",opt.last_optimize_result())

minimum at [ 3.14159265e+00  2.22144147e+00 -7.51315101e-11  2.22144147e+00]
the objective function converges to 5.858023705462033e-16
optimization result:  1


In [ ]:
logging.getLogger('tensorflow').setLevel(logging.ERROR) # quiets Tensorflow warnings.
obj_sk = generate_obj_sk(h_gate) # generates an objective function for a target unitary matrix.
opt = nlopt.opt(nlopt.LD_LBFGS,16) # chooses an optimization algorithm.
opt.set_min_objective(obj_sk) # sets the objective function for the chosen optimization algorithm.
opt.set_xtol_rel(1e-5) # sets the tolerance for the convergence.
theta_opt_sk = opt.optimize(np.ones(16)) # sets the initial varaibles.
obj_final = opt.last_optimum_value() # assigns the variables of the last iteration.
# prints results of the optimization algorithm
print("minimum at",theta_opt_sk)
print("the objective function converges to", obj_final)
print("optimization result: ",opt.last_optimize_result())

minimum at [3.14159265 1.57079632 1.5707963  1.57079632 1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.        ]
the objective function converges to 9.110270824146652e-16
optimization result:  4


In [ ]:
# validation
eu1 = Euler(1)
eu2 = Euler(2)
fk_mat_su2 = generate_first_kind(eu1)
fk_mat_su4 = generate_first_kind(eu2)
sk_mat_su2 = generate_second_kind(eu1)
sk_mat_su4 = generate_second_kind(eu2)

In [ ]:
fk_mat_su2(theta_opt_fk)

<tf.Tensor: shape=(2, 2), dtype=complex128, numpy=
array([[ 0.70710678+2.39601672e-11j,  0.70710678+9.63573665e-12j],
       [ 0.70710678+5.74658654e-11j, -0.70710678-4.31413794e-11j]])>

In [ ]:
unitary_target = cnot_gate
dim = tf.reduce_prod(unitary_target.shape).numpy().astype(int).item()

In [ ]:
dim

16

In [ ]:
logging.getLogger('tensorflow').setLevel(logging.ERROR) # quiets Tensorflow warnings.
obj_fk = generate_obj_fk(unitary_target) # generates an objective function for a target unitary matrix.
opt = nlopt.opt(nlopt.LD_LBFGS,dim) # chooses an optimization algorithm.
opt.set_min_objective(obj_fk) # sets the objective function for the chosen optimization algorithm.
opt.set_xtol_rel(1e-5) # sets the tolerance for the convergence.
theta_opt_fk = opt.optimize(np.ones(dim)) # sets the initial varaibles.
obj_final = opt.last_optimum_value() # assigns the variables of the last iteration.
# prints results of the optimization algorithm
print("minimum at",theta_opt_fk)
print("the objective function converges to", obj_final)
print("optimization result: ",opt.last_optimize_result())

minimum at [ 1.57079634e+00  1.57079631e+00  7.08445492e-09  7.08445507e-09
  7.08445500e-09 -1.69169595e-08 -2.08570818e-09 -2.32231501e-08
  7.08445500e-09 -1.69169595e-08 -2.32231501e-08 -2.08570818e-09
  1.57079631e+00  1.57079629e+00 -1.69169596e-08 -1.69169598e-08]
the objective function converges to 3.436238964984065e-15
optimization result:  4


In [ ]:
logging.getLogger('tensorflow').setLevel(logging.ERROR) # quiets Tensorflow warnings.
obj_sk = generate_obj_sk(h_gate) # generates an objective function for a target unitary matrix.
opt = nlopt.opt(nlopt.LD_LBFGS,16) # chooses an optimization algorithm.
opt.set_min_objective(obj_sk) # sets the objective function for the chosen optimization algorithm.
opt.set_xtol_rel(1e-5) # sets the tolerance for the convergence.
theta_opt_sk = opt.optimize(np.ones(16)) # sets the initial varaibles.
obj_final = opt.last_optimum_value() # assigns the variables of the last iteration.
# prints results of the optimization algorithm
print("minimum at",theta_opt_sk)
print("the objective function converges to", obj_final)
print("optimization result: ",opt.last_optimize_result())

minimum at [3.14159265 1.57079632 1.5707963  1.57079632 1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.        ]
the objective function converges to 9.110270824146652e-16
optimization result:  4


In [ ]:
# validation
eu1 = Euler(2)
eu2 = Euler(4)
fk_mat_su2 = generate_first_kind(eu1)
fk_mat_su4 = generate_first_kind(eu2)
sk_mat_su2 = generate_second_kind(eu1)
sk_mat_su4 = generate_second_kind(eu2)

In [ ]:
np.round(fk_mat_su2(theta_opt_fk),6)

array([[ 1.-0.j, -0.+0.j, -0.+0.j, -0.+0.j],
       [ 0.+0.j,  1.-0.j,  0.+0.j, -0.+0.j],
       [ 0.+0.j,  0.+0.j, -0.-0.j,  1.-0.j],
       [ 0.+0.j, -0.+0.j,  1.+0.j,  0.-0.j]])